In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [7]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
niter = 500
rank = 80
tau = 5
alpha = 1
gamma = 0.5
rho = 0.5

#alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 = alpha8 = alpha9 = alpha10 = alpha
#for rank in (35,): #(25,30,35)(100,180,220,260,300,340,380)(20,60,100,140,180,220,260,300,340,380)
#    for tau in (400,500): #(300,400,500)(10,50,100,200,300)(10,50,100,200,300)
#        for gamma in (0.5,0.8,2): #(0.5,0.8,2)(0.5,0.8)(0.5,1,1.5,2,3)
            #gamma1 = gamma2 = gamma3 = gamma4 = gamma5 = gamma
#            for rho in (0.5,0.8,2): #(0.5,0.8)(1,2)
                #rho1 = rho2 = rho3 = rho4 = rho5= rho
#                for alpha in (0.5,1,1.5,2):
#                    print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)
                    #print('Compression Ratio', ((1024*28*28+10*1024+(8*(rank)+32*np.square(rank))*2)/(1024*28*28+10*1024+1024*1024*2)), (8*(rank)+32*np.square(rank))*2/(1024*1024*2))
        
print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d0 = 561 #561 =3*11*17
    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 6 

    W1 = 0.2*init.orthogonal_(torch.empty(d1, d0, device=device), gain=1.0)
    #W1 = 0.01*torch.randn(d1, d0, device=device)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = 0.2*init.orthogonal_(torch.empty(d2, d1, device=device), gain=1.0)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = 0.2*init.orthogonal_(torch.empty(d3, d2, device=device), gain=1.0)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = 0.2*init.orthogonal_(torch.empty(d4, d3, device=device), gain=1.0)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)


    W5 = 0.2*init.orthogonal_(torch.empty(d5, d4, device=device), gain=1.0)
    b5 = 0*torch.ones(d5, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = U5 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V5 = (y_one_hot + gamma*U5 + alpha*V5)/(1 + gamma + alpha)

        # update U4 
        U5 = (gamma*V5 + rho*(torch.mm(W5,V4) + b5.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W5, b5 = updateWb_org(U5,V4,W5,b5,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)



  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b5.repeat(1, N), W5, a4_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        pred_test = torch.argmax(torch.addmm(b5.repeat(1, N_test), W5, a4_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V5,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,U5,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4

    CR_1=((total_variabels)+(d4*d5))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#this postion to add new row into existing table
    #df=pd.read_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
    #new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
    #           'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
    #           'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'seed':seed} 
    #df=df.append(new_row,ignore_index=True)
    #df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv',index=False)


filename= "Orthogonal_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/4 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 5 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 1.3331913948059082 - sq_loss: 661.6790161132812 - tot_loss: 943.2722693939868 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 2 / 500 
 - time: 1.3223369121551514 - sq_loss: 294.0794372558594 - tot_loss: 472.80204450618476 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 3 / 500 
 - time: 1.2913367748260498 - sq_loss: 161.7793426513672 - tot_loss: 258.726295045577 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 4 / 500 
 - time: 1.3005797863006592 - sq_loss: 87.77423858642578 - tot_loss: 144.54296143585816 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 5 / 500 
 - time: 1.3040597438812256 - sq_loss: 47.226661682128906 - tot_loss: 83.07219670014456 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 6 / 500 
 - time: 1.3

Repeatition 1 Epoch 48 / 500 
 - time: 1.2974371910095215 - sq_loss: 0.0005230962415225804 - tot_loss: 0.7959144891920005 - acc: 0.8841131664853101 - val_acc: 0.8788598574821853
Repeatition 1 Epoch 49 / 500 
 - time: 1.293478012084961 - sq_loss: 0.00048597133718430996 - tot_loss: 0.7622168185862392 - acc: 0.8861534276387377 - val_acc: 0.8815744825246012
Repeatition 1 Epoch 50 / 500 
 - time: 1.2952661514282227 - sq_loss: 0.0004520008515100926 - tot_loss: 0.7273503222295403 - acc: 0.8877856365614799 - val_acc: 0.8839497794367153
Repeatition 1 Epoch 51 / 500 
 - time: 1.2909834384918213 - sq_loss: 0.00042081388528458774 - tot_loss: 0.6920428136490955 - acc: 0.888873775843308 - val_acc: 0.8849677638276213
Repeatition 1 Epoch 52 / 500 
 - time: 1.2959933280944824 - sq_loss: 0.0003921631723642349 - tot_loss: 0.6588754977183271 - acc: 0.8921381936887922 - val_acc: 0.8893790295215473
Repeatition 1 Epoch 53 / 500 
 - time: 1.294593095779419 - sq_loss: 0.00036574737168848515 - tot_loss: 0.63200

Repeatition 1 Epoch 94 / 500 
 - time: 1.2817730903625488 - sq_loss: 3.9764669054420665e-05 - tot_loss: 0.18314714720577285 - acc: 0.9515778019586507 - val_acc: 0.9450288428910757
Repeatition 1 Epoch 95 / 500 
 - time: 1.3009538650512695 - sq_loss: 3.814569572568871e-05 - tot_loss: 0.1775559585485098 - acc: 0.9525299238302503 - val_acc: 0.9450288428910757
Repeatition 1 Epoch 96 / 500 
 - time: 1.2975265979766846 - sq_loss: 3.66355488949921e-05 - tot_loss: 0.17501370633237912 - acc: 0.9526659412404788 - val_acc: 0.9446895147607737
Repeatition 1 Epoch 97 / 500 
 - time: 1.2846431732177734 - sq_loss: 3.5188197216484696e-05 - tot_loss: 0.17607703132489405 - acc: 0.9533460282916213 - val_acc: 0.9453681710213777
Repeatition 1 Epoch 98 / 500 
 - time: 1.2892775535583496 - sq_loss: 3.3842665288830176e-05 - tot_loss: 0.17028111716763306 - acc: 0.954570184983678 - val_acc: 0.9457074991516796
Repeatition 1 Epoch 99 / 500 
 - time: 1.291792631149292 - sq_loss: 3.255652336520143e-05 - tot_loss: 0.1

Repeatition 1 Epoch 140 / 500 
 - time: 1.280820369720459 - sq_loss: 1.1571272807486821e-05 - tot_loss: 0.10020506069048452 - acc: 0.9776931447225244 - val_acc: 0.9548693586698337
Repeatition 1 Epoch 141 / 500 
 - time: 1.285393476486206 - sq_loss: 1.1409703802200966e-05 - tot_loss: 0.10045466485048848 - acc: 0.977829162132753 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 142 / 500 
 - time: 1.2923226356506348 - sq_loss: 1.1252298463659827e-05 - tot_loss: 0.10173223624059347 - acc: 0.977829162132753 - val_acc: 0.9548693586698337
Repeatition 1 Epoch 143 / 500 
 - time: 1.2882411479949951 - sq_loss: 1.1111275853181724e-05 - tot_loss: 0.09966430079020938 - acc: 0.9779651795429815 - val_acc: 0.9548693586698337
Repeatition 1 Epoch 144 / 500 
 - time: 1.2930469512939453 - sq_loss: 1.0978259524563327e-05 - tot_loss: 0.10011600288333966 - acc: 0.9782372143634385 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 145 / 500 
 - time: 1.2838518619537354 - sq_loss: 1.0842535630217753e-05 - tot_

Repeatition 1 Epoch 186 / 500 
 - time: 1.2878367900848389 - sq_loss: 7.78604135120986e-06 - tot_loss: 0.07918836919031236 - acc: 0.984766050054407 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 187 / 500 
 - time: 1.2904002666473389 - sq_loss: 7.747692507109605e-06 - tot_loss: 0.07635928426585536 - acc: 0.984766050054407 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 188 / 500 
 - time: 1.294083595275879 - sq_loss: 7.710049430897925e-06 - tot_loss: 0.08131529409485694 - acc: 0.9849020674646355 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 189 / 500 
 - time: 1.280777931213379 - sq_loss: 7.667755198781379e-06 - tot_loss: 0.0802311862949523 - acc: 0.9849020674646355 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 190 / 500 
 - time: 1.2835814952850342 - sq_loss: 7.630664185853675e-06 - tot_loss: 0.08104189621109015 - acc: 0.9849020674646355 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 191 / 500 
 - time: 1.2861683368682861 - sq_loss: 7.596946488774847e-06 - tot_loss: 0.

...

Repeatition 1 Epoch 278 / 500 
 - time: 1.2865221500396729 - sq_loss: 5.621262971544638e-06 - tot_loss: 0.06351738006273067 - acc: 0.9877584330794341 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 279 / 500 
 - time: 1.2923519611358643 - sq_loss: 5.608198534901021e-06 - tot_loss: 0.06707395941782224 - acc: 0.9877584330794341 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 280 / 500 
 - time: 1.2828426361083984 - sq_loss: 5.595087714027613e-06 - tot_loss: 0.06325055955327485 - acc: 0.9877584330794341 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 281 / 500 
 - time: 1.2790799140930176 - sq_loss: 5.578395303018624e-06 - tot_loss: 0.06446597605654603 - acc: 0.9877584330794341 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 282 / 500 
 - time: 1.2809619903564453 - sq_loss: 5.563161266763927e-06 - tot_loss: 0.06233799701770337 - acc: 0.9877584330794341 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 283 / 500 
 - time: 1.2927117347717285 - sq_loss: 5.548332410398871e-06 - tot_lo

Repeatition 1 Epoch 324 / 500 
 - time: 1.306692361831665 - sq_loss: 5.026561211707303e-06 - tot_loss: 0.06210401317309078 - acc: 0.9885745375408053 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 325 / 500 
 - time: 1.2863340377807617 - sq_loss: 5.014022917748662e-06 - tot_loss: 0.06266945833147552 - acc: 0.9887105549510338 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 326 / 500 
 - time: 1.2888412475585938 - sq_loss: 5.002545094612287e-06 - tot_loss: 0.06247971116766671 - acc: 0.9889825897714908 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 327 / 500 
 - time: 1.2806313037872314 - sq_loss: 4.990286925021792e-06 - tot_loss: 0.05815366445472314 - acc: 0.9889825897714908 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 328 / 500 
 - time: 1.2758073806762695 - sq_loss: 4.9783166105044074e-06 - tot_loss: 0.0600640682823137 - acc: 0.9889825897714908 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 329 / 500 
 - time: 1.28609299659729 - sq_loss: 4.9673603825795e-06 - tot_loss: 0

Repeatition 1 Epoch 370 / 500 
 - time: 1.2907483577728271 - sq_loss: 4.554438874038169e-06 - tot_loss: 0.06082400628305784 - acc: 0.9902067464635473 - val_acc: 0.9616559212758737
Repeatition 1 Epoch 371 / 500 
 - time: 1.2772095203399658 - sq_loss: 4.544896455627168e-06 - tot_loss: 0.05845234843004121 - acc: 0.9902067464635473 - val_acc: 0.9616559212758737
Repeatition 1 Epoch 372 / 500 
 - time: 1.27708101272583 - sq_loss: 4.533220817393158e-06 - tot_loss: 0.06513665733461771 - acc: 0.9902067464635473 - val_acc: 0.9619952494061758
Repeatition 1 Epoch 373 / 500 
 - time: 1.285243034362793 - sq_loss: 4.52443464382668e-06 - tot_loss: 0.05751363699470957 - acc: 0.9902067464635473 - val_acc: 0.9619952494061758
Repeatition 1 Epoch 374 / 500 
 - time: 1.2842607498168945 - sq_loss: 4.515912678471068e-06 - tot_loss: 0.05829417731774278 - acc: 0.9902067464635473 - val_acc: 0.9619952494061758
Repeatition 1 Epoch 375 / 500 
 - time: 1.2767436504364014 - sq_loss: 4.507565336098196e-06 - tot_loss: 

Repeatition 1 Epoch 416 / 500 
 - time: 1.2843372821807861 - sq_loss: 4.172089120402234e-06 - tot_loss: 0.05448649494116786 - acc: 0.9904787812840044 - val_acc: 0.9640312181879878
Repeatition 1 Epoch 417 / 500 
 - time: 1.2908227443695068 - sq_loss: 4.1657776819192804e-06 - tot_loss: 0.061620488290444086 - acc: 0.9904787812840044 - val_acc: 0.9636918900576857
Repeatition 1 Epoch 418 / 500 
 - time: 1.2922987937927246 - sq_loss: 4.157535840931814e-06 - tot_loss: 0.05660546280486578 - acc: 0.9904787812840044 - val_acc: 0.9643705463182898
Repeatition 1 Epoch 419 / 500 
 - time: 1.2900912761688232 - sq_loss: 4.147246272623306e-06 - tot_loss: 0.058020849463760626 - acc: 0.9904787812840044 - val_acc: 0.9643705463182898
Repeatition 1 Epoch 420 / 500 
 - time: 1.2756767272949219 - sq_loss: 4.139929387747543e-06 - tot_loss: 0.055881563891423625 - acc: 0.9904787812840044 - val_acc: 0.9647098744485918
Repeatition 1 Epoch 421 / 500 
 - time: 1.2841112613677979 - sq_loss: 4.134577011427609e-06 - to

Repeatition 1 Epoch 462 / 500 
 - time: 1.2916502952575684 - sq_loss: 3.85966086469125e-06 - tot_loss: 0.05386531326490562 - acc: 0.9907508161044614 - val_acc: 0.9660671869697998
Repeatition 1 Epoch 463 / 500 
 - time: 1.2775061130523682 - sq_loss: 3.853873749903869e-06 - tot_loss: 0.058550376416366845 - acc: 0.9907508161044614 - val_acc: 0.9664065151001018
Repeatition 1 Epoch 464 / 500 
 - time: 1.2965831756591797 - sq_loss: 3.845966602966655e-06 - tot_loss: 0.05880224189412431 - acc: 0.9907508161044614 - val_acc: 0.9664065151001018
Repeatition 1 Epoch 465 / 500 
 - time: 1.2930970191955566 - sq_loss: 3.840902991214534e-06 - tot_loss: 0.05661751615064503 - acc: 0.9907508161044614 - val_acc: 0.9667458432304038
Repeatition 1 Epoch 466 / 500 
 - time: 1.295882225036621 - sq_loss: 3.832974471151829e-06 - tot_loss: 0.05270220627759947 - acc: 0.9907508161044614 - val_acc: 0.9664065151001018
Repeatition 1 Epoch 467 / 500 
 - time: 1.2873046398162842 - sq_loss: 3.827164618996903e-06 - tot_los